<a href="https://colab.research.google.com/github/dung24mse13190/mle-final-new/blob/main/DungTNT_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
df_train = pd.read_csv('/kaggle/input/healthcare-dataset-stroke-data/healthcare-dataset-stroke-data.csv')

# Display the first 5 rows of the dataframe
display(df_train.head())

# Display the shape of the dataframe
print(f"Shape of the dataframe: {df_train.shape}")